In [25]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('vader_lexicon')
class SeleniumClient(object):
    def __init__(self):
        #Initialization method. 
        self.chrome_options = webdriver.ChromeOptions()
        self.chrome_options.add_argument('--headless')
        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-setuid-sandbox')
        self.chrome_options.add_argument("start-maximized")
        self.chrome_options.add_argument("disable-infobars")
        self.chrome_options.add_argument("--disable-extensions")

        # you need to provide the path of chromdriver in your system
    

        chromedriver = "C:/Users/ManasiAdmin/chromedriver"
        os.environ["webdriver.chrome.driver"] = chromedriver
       
        
        self.driver = webdriver.Chrome(chromedriver)
        #driver.get("https://twitter.com/search?q=amex")
        #time.sleep(10)
        #driver.quit()

        
        #self.browser = webdriver.Chrome('C:/Users/ManasiAdmin/chromedriver', options=self.chrome_options)

        self.base_url = 'https://twitter.com/search?q='

    def get_tweets(self, query):
        ''' 
        Function to fetch tweets. 
        '''
        #try: 
        self.driver.get(self.base_url+query)
        time.sleep(10)

        body = self.driver.find_element_by_tag_name('body')

        for _ in range(500):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)

        timeline = self.driver.find_element_by_id('timeline')
        #user_nodes = timeline.find_element_by_css_selector("h1.ProfileHeaderCard-name a.ProfileHeaderCard-nameLink")
        tweet_nodes = timeline.find_elements_by_css_selector('.tweet-text')
            #tweet_nodes = timeline.find_elements_by_css_selector('[id^=\".tweet-text\"]')

        return pd.DataFrame({'tweets': [tweet_node.text for tweet_node in tweet_nodes] ,'tweet_id': [tweet_node.id for tweet_node in tweet_nodes]})

        #except: 
                #print("Selenium - An error occured while fetching tweets.")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ManasiAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [43]:
selenium_client = SeleniumClient()

tweets_df = selenium_client.get_tweets('americanexpress')

In [44]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob 
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    if polarity_scores['neg'] > polarity_scores['pos']:
        return 'negative'
    else:
        return 'positive'

def fetch_sentiment_using_textblob(text):
    analysis = TextBlob(text)
    # set sentiment 
    if analysis.sentiment.polarity >= 0:
        return 'positive'
    else: 
        return 'negative'

In [45]:
sentiments_using_textblob = tweets_df.tweets.apply(lambda tweet: fetch_sentiment_using_textblob(tweet))
pd.DataFrame(sentiments_using_textblob.value_counts())

,tweets
positive,555
negative,74


In [46]:
sentiments_using_SIA = tweets_df.tweets.apply(lambda tweet: fetch_sentiment_using_SIA(tweet))
pd.DataFrame(sentiments_using_SIA.value_counts())

,tweets
positive,541
negative,88


In [47]:
#We get less biased results using textblob
tweets_df['sentiment'] = sentiments_using_textblob
tweets_df

,tweets,tweet_id,sentiment
0,The research found that the top three areas of...,0.9004008482888513-3,positive
1,Now booking #ELANTRA and #TUCSON has become ea...,0.9004008482888513-4,positive
2,"Word of advice, don’t add your AMEX card to De...",0.9004008482888513-5,positive
3,@AmexIndia @AmericanExpress \n\nYOUR HELPDESK ...,0.9004008482888513-6,positive
4,What is this unsolicited mail...calling custom...,0.9004008482888513-7,positive
5,American Express breaks ground on New York JFK...,0.9004008482888513-8,positive
6,Ask America express I am trying to cancel my d...,0.9004008482888513-9,positive
7,"@AmericanExpress stop calling me, please!!",0.9004008482888513-10,positive
8,"Yesterday, #PAWJourney visited with American E...",0.9004008482888513-11,positive
9,Congrats to Kostas Antetokounmpo (@Kostas_Ant...,0.9004008482888513-12,positive


In [48]:
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    
    return text
# We are keeping cleaned tweets in a new column called 'tidy_tweets'
tweets_df['tidy_tweets'] = np.vectorize(remove_pattern)(tweets_df['tweets'], "@[\w]*: | *RT*")

In [49]:
cleaned_tweets = []

for index, row in tweets_df.iterrows():
    # Here we are filtering out all the words that contains link
    words_without_links = [word for word in row.tidy_tweets.split()        if 'http' not in word]
    cleaned_tweets.append(' '.join(words_without_links))

tweets_df['tidy_tweets'] = cleaned_tweets

In [50]:
tweets_df.drop_duplicates(subset=['tidy_tweets'], keep=False)

,tweets,tweet_id,sentiment,tidy_tweets
0,The research found that the top three areas of...,0.9004008482888513-3,positive,The research found that the top three areas of...
1,Now booking #ELANTRA and #TUCSON has become ea...,0.9004008482888513-4,positive,Now booking #ELANTA and #TUCSON has become eas...
2,"Word of advice, don’t add your AMEX card to De...",0.9004008482888513-5,positive,"Word of advice, don’t add your AMEX card to De..."
3,@AmexIndia @AmericanExpress \n\nYOUR HELPDESK ...,0.9004008482888513-6,positive,@AmexIndia @AmericanExpress YOU HELPDESK PAGE ...
4,What is this unsolicited mail...calling custom...,0.9004008482888513-7,positive,What is this unsolicited mail...calling custom...
5,American Express breaks ground on New York JFK...,0.9004008482888513-8,positive,American Express breaks ground on New York JFK...
6,Ask America express I am trying to cancel my d...,0.9004008482888513-9,positive,Ask America express I am trying to cancel my d...
7,"@AmericanExpress stop calling me, please!!",0.9004008482888513-10,positive,"@AmericanExpress stop calling me, please!!"
8,"Yesterday, #PAWJourney visited with American E...",0.9004008482888513-11,positive,"Yesterday, #PAWJourney visited with American E..."
9,Congrats to Kostas Antetokounmpo (@Kostas_Ant...,0.9004008482888513-12,positive,Congrats to Kostas Antetokounmpo (@Kostas_Ante...


In [52]:
#Extracting those id's where sentiment is negative (dissatisfied)
tweets_df.loc[tweets_df['sentiment'] == 'negative']

,tweets,tweet_id,sentiment,tidy_tweets
11,Highlights from my speaking engagement at Amer...,0.9004008482888513-14,negative,Highlights from my speaking engagement at Amer...
17,Fuck the American Express Card\nI never leave ...,0.9004008482888513-20,negative,Fuck the American Express Card I never leave h...
18,#Stonegasmoviechallenge2019 (MARCH 18: MOVIE W...,0.9004008482888513-21,negative,#Stonegasmoviechallenge2019 (MACH 18: MOVIE WI...
21,.@AmericanExpress Card Members! Stop by sectio...,0.9004008482888513-24,negative,.@AmericanExpress Card Members! Stop by sectio...
23,I know twitter rants about shitty companies is...,0.9004008482888513-26,negative,I know twitter rants about shitty companies is...
24,Dow-member American Express $AXP just recently...,0.9004008482888513-27,negative,Dow-member American Express $AXP just recently...
27,Del Taco does not accept American Express. WTF?,0.9004008482888513-30,negative,Del Taco does not accept American Express. WTF?
47,American Express Sending Out Checks for Missin...,0.9004008482888513-50,negative,American Express Sending Out Checks for Missin...
48,#Isles legend Bryan Trottier is in the house t...,0.9004008482888513-51,negative,#Isles legend Bryan Trottier is in the house t...
54,Fuck you @AmericanExpress pic.twitter.com/xUQj...,0.9004008482888513-57,negative,Fuck you @AmericanExpress pic.twitter.com/xUQj...


In [53]:
#Removing punctuations numbers and special characters
tweets_df['absolute_tidy_tweets'] = tweets_df['tidy_tweets'].str.replace("[^a-zA-Z# ]", "")

In [54]:
#Removing stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_set = set(stopwords.words("english"))
cleaned_tweets = []

for index, row in tweets_df.iterrows():
    
    # filerting out all the stopwords 
    words_without_stopwords = [word for word in row.absolute_tidy_tweets.split() if not word in stopwords_set]
    
    # finally creating tweets list of tuples containing stopwords(list) and sentimentType 
    cleaned_tweets.append(' '.join(words_without_stopwords))

tweets_df['absolute_tidy_tweets'] = cleaned_tweets

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ManasiAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
from nltk.stem import WordNetLemmatizer
# Tokenization
tokenized_tweet = tweets_df['absolute_tidy_tweets'].apply(lambda x: x.split())
# Finding Lemma for each word
word_lemmatizer = WordNetLemmatizer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [word_lemmatizer.lemmatize(i) for i in x])
#joining words into sentences (from where they came from)
for i, tokens in enumerate(tokenized_tweet):
    tokenized_tweet[i] = ' '.join(tokens)

tweets_df['absolute_tidy_tweets'] = tokenized_tweet

In [56]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# BOW features
bow_word_vectorizer = CountVectorizer(max_df=0.90, min_df=2, stop_words='english')
# bag-of-words feature matrix
bow_word_feature = bow_word_vectorizer.fit_transform(tweets_df['absolute_tidy_tweets'])

# TF-IDF features
tfidf_word_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, stop_words='english')
# TF-IDF feature matrix
tfidf_word_feature = tfidf_word_vectorizer.fit_transform(tweets_df['absolute_tidy_tweets'])

In [57]:
target_variable = tweets_df['sentiment'].apply(lambda x: 0 if x=='negative' else 1 )

In [58]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
def naive_model(X_train, X_test, y_train, y_test):
    naive_classifier = GaussianNB()
    naive_classifier.fit(X_train.toarray(), y_train)

    # predictions over test set
    predictions = naive_classifier.predict(X_test.toarray())
    
    # calculating f1 score
    print(f'F1 Score - {f1_score(y_test, predictions)}')

In [59]:
X_train, X_test, y_train, y_test = train_test_split(bow_word_feature, target_variable, test_size=0.3, random_state=870)
naive_model(X_train, X_test, y_train, y_test)

F1 Score - 0.9294117647058823
